In [32]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import sys
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Import from function

In [33]:
%load_ext autoreload
%autoreload 2
PATH = os.path.dirname(os.getcwd()) 
sys.path.append(PATH)
df = pd.read_csv(PATH + '/data/cleaned_data.csv')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from skills_matcher.scripts.spacy_model import json_to_train_data, split_data, make_spacy_model, load_results
training_data = json_to_train_data(PATH + '/notebooks/annotation_data.json')
X_train, X_val = split_data(training_data, val_split=0.2)
spacy_train = make_spacy_model(X_train)
spacy_train.to_disk('train.spacy')
spacy_val = make_spacy_model(X_val)
spacy_val.to_disk('val.spacy')

 88%|████████████████████████████████████████████████████████████████████████████████████▍           | 51/58 [00:00<00:00, 98.79it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 67.37it/s]

Skipping entity
Skipping entity


In [6]:
! python -m spacy init fill-config base_config.cfg config.cfg

2021-12-03 14:58:52.285589: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-03 14:58:52.286641: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [7]:
! python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./train.spacy

2021-12-03 14:59:19.199775: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-03 14:59:19.199845: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2021-12-03 14:59:22,505] [INFO] Set up nlp object from config
[2021-12-03 14:59:22,531] [INFO] Pipeline: ['tok2vec', 'ner']
[2021-12-03 14:59:22,540] [INFO] Created vocabulary
[2021-12-03 14:59:22,542] [INFO] Finished initializing nlp object
[2021-12-03 14:59:25,719] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2v

In [35]:
text = df.description[124]
entities = load_results(PATH + '/notebooks/output/model-best', text)